# Bikesharing in Seoul
* https://archive.ics.uci.edu/ml/datasets/Seoul+Bike+Sharing+Demand

------------------------

## Data cleaning

 1. Normalize column names
 2. Change type of column `date` to **datetime64**
 3. Change values of table `holiday` from **Holiday**/**No Holiday** to **Yes**/**No**.
 4. Create column `daytime` with values **Morning, Noon, Afternoon, Evening, Night**.
 5. Create column `temperaure_type` with values: **Hot, Warm, Mild, Cold, Frost**.


#### Columns after cleaning

|Column Name|Datatype|Type|Values|
|:----------|:-------|:---|:-----|
|date|datetime64|categorical|2017/12/1 - 2018/11/30|
|rented_bike_count|int64|numerical|0 - 3556|
|hour|int64|numerical|0 - 23|
|temperature|float64|numerical|-17.8 - 39.4|
|humidity|int64|numerical|0.0 - 98.0|
|wind_speed|float64|numerical|0.0 - 7.4|
|visibility|int64|numeric|27-2000|
|dew_point_temperature|float64|numerical|-30.6 - 27.2|
|solar_radiation|float64|numerical|0.0 - 3.52|
|rainfall|float|numerical|0.0 - 35.0|
|snowfall|float|numerical|0.0 - 8.0|
|seasons|object|categorical|Spring,Summer,Autumn,Winter|
|holiday|object|categorical|Yes,No|
|functioning_day|object|categorical|Yes,No|
|daytime|object|categorical|Morning,Noon,Afternoon,Evening,Night|
|temperature_type|object|categorical|Hot,Warm,Mild,Cold,Frost|

In [1]:
import pandas as pd
import numpy as npb
import seaborn as sns
import yaml
import re

In [2]:
cmt='''def show_unique_values(df):
    for col in df.columns:
        print(f"COLUMN '{col}' ({df[col].dtype})")
        print(df[col].unique())
        print()
        
show_unique_values(df)'''

In [3]:
def normalize_column_names(columns):
    '''
    Normalize column names.

    The following transformations will be applied:
    
    - Remove parenthesis and their content
    - Convert all letters to lower case
    - Replace whitespaces with underlines (' ' -> '_')
    - Remove leading/trailing whitespaces, tabs and newlines

    Args:
        columns: List of column names
    Return:
        List of normalized columns
    '''
    norm_columns = []
    for col in columns:
        # Remove parenthesis and their content
        colstr = re.sub('\(.*\)', '', col)
        # To lower case letters, replace ' ' to '_' and
        # remove leading/trailing whitespaces
        colstr = colstr.strip().lower().replace(' ', '_')
        norm_columns.append(colstr)

    return norm_columns

In [4]:
def adjust_column_types(data):
    '''
    Change datatype of column "date" to pd.datetime64.

    Args:
        data: Dataframe
    Return:
        Cleaned dataframe
    '''
    df = data.copy()
    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y', errors='coerce')    
    return df

In [5]:
def change_holiday_values(data):
    '''
    Changes values of the column holiday.
    
      "Holiday"    -> "Yes"
      "No Holiday" -> "No"

    Args:
        data: Dataset to clean
    Return:
        Cleaned dataset
    '''
    df = data.copy()
    df['holiday'] = df['holiday'].replace({'Holiday':'Yes', 'No Holiday':'No'})
    return df

In [6]:
def add_daytime_column(data):
    '''
    Creates a new column "daytime" depending on column "hour".
    Values of the new column can be Morning, Noon, Afternoon, Evening or Night.

      hour 4 to 10  -> Morning
      hour 11 to 14 -> Noon
      hour 15 to 17 -> Afternoon
      hour 18 to 21 -> Evening
      hour 22 to 3  -> Night

    Args:
        data: Dataframe to add column to
    Return:
        New dataframe
    '''
    def get_time_of_day(hour):
        if hour >= 4 and hour < 11:
            return "Morning"
        elif hour >= 11 and hour < 15:
            return "Noon"
        elif hour >= 15 and hour < 18:
            return "Afternoon"
        elif hour >= 18 and hour < 22:
            return "Evening"
        else:
            return "Night"
    df = data.copy()
    df['daytime'] = df['hour'].apply(get_time_of_day)
    return df

In [7]:
def add_temperature_type_column(data):
    '''
    Creates a new column "temperature_type" depending on column "temperature".
    Values of the new column can be Hot, Warm, Mild, Cold or Frost.

      temp > 27  -> Hot
      temp 19-27 -> Warm
      temp 10-19 -> Mild
      temp 0-10  -> Cold
      temp < 0   -> Frost

    Args:
        data: Dataframe to add column to
    Return:
        New dataframe
    '''   
    def get_temperature_type(temp):
        if temp > 27:
            return "Hot"
        elif temp > 19 and temp <= 27:
            return "Warm"
        elif temp > 10 and temp <= 19:
            return "Mild"
        elif temp >= 0 and temp <= 10:
            return "Cold"
        else:
            return "Frost"
    df = data.copy()
    df['temperature_type'] = df['temperature'].apply(get_temperature_type)
    return df

In [8]:
def clean_data(data):
    '''
    Clean the total bike sharing dataset.
    
    Args:
        data: Dataset to clean
    Return:
        Cleaned dataset
    '''
    
    df = data.copy()
    df.columns = normalize_column_names(df)
    df = adjust_column_types(df)
    df = change_holiday_values(df)
    df = add_daytime_column(df)
    df = add_temperature_type_column(df)

    return df

#### Open YAML config file

In [9]:
# Open yaml configs
with open('../params.yaml') as file:
    config = yaml.safe_load(file)

#### Load dataset

In [10]:
df = pd.read_csv(config['data']['csv_raw'])

#### Clean data

In [11]:
df_clean = clean_data(df)

#### Store cleaned data to file

In [12]:
df_clean.to_csv(config['data']['csv_cleaned'], index=False)

#### Check cleaned data

In [13]:
df_clean.isna().sum().sum()

0

In [14]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   8760 non-null   datetime64[ns]
 1   rented_bike_count      8760 non-null   int64         
 2   hour                   8760 non-null   int64         
 3   temperature            8760 non-null   float64       
 4   humidity               8760 non-null   int64         
 5   wind_speed             8760 non-null   float64       
 6   visibility             8760 non-null   int64         
 7   dew_point_temperature  8760 non-null   float64       
 8   solar_radiation        8760 non-null   float64       
 9   rainfall               8760 non-null   float64       
 10  snowfall               8760 non-null   float64       
 11  seasons                8760 non-null   object        
 12  holiday                8760 non-null   object        
 13  fun